In [1]:
using PCT

## $x^{\dagger} A x$

In [2]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j))); f

(A) -> 
    𝒫((x) -> 
        ∑((i, j), (x(i)'⋅x(j)⋅A(i, j))))

In [3]:
df = vdiff(f; settings=symmetry_settings)

(A) -> 
    (x) -> 
        (_d) -> 
            ∑((j), (2.0⋅x(j)⋅A(_d, j)))

## Trace of Product

In [4]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::Z, j::Z) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [5]:
vdiff(eval_all(f); settings=symmetry_settings)

(A, B) -> 
    (_d, _d_1) -> 
        (2.0⋅B(_d_1, _d)'), (_d, _d_1) -> 
        (2.0⋅A(_d_1, _d)')

## Conjugate Gradient

In [6]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::Z) -> x(i) + α * (r(i) + β * p(i))))
end; f

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (0.5⋅x(i)⋅x(j)⋅A(i, j))))
        𝒫((α, β) -> 
            E((i) -> 
                ((α⋅((β⋅p(i))+r(i)))+x(i))))
    end

In [7]:
df = vdiff(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (0.5⋅x(i)⋅x(j)⋅A(i, j))))
        (α, β) -> 
            ∑((i), (((β⋅p(i))+r(i))⋅𝒫(E)((i) -> 
                ((α⋅((β⋅p(i))+r(i)))+x(i)), 1)(i))), ∑((i), (α⋅p(i)⋅𝒫(E)((i) -> 
                ((α⋅((β⋅p(i))+r(i)))+x(i)), 1)(i)))
    end

In [8]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i), (((β⋅p(i))+r(i))⋅b(i)))+∑((i, j), (((α⋅((β⋅p(j))+r(j)))+x(j))⋅((β⋅p(i))+r(i))⋅A(i, j)))), (∑((i), (α⋅b(i)⋅p(i)))+∑((i, j), (α⋅((α⋅((β⋅p(j))+r(j)))+x(j))⋅p(i)⋅A(i, j))))

## Lagrangian Mechanics

In [9]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [10]:
f = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [11]:
vdiff(f; settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        ((_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1)+D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1)))

## Hartree Fock Gradient and Hessian Vector Product

In [12]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [13]:
f = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s)))

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))

In [14]:
df = vdiff(f; settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            ∑((i, p, q, r), (4.0⋅C(q, i)'⋅J(_d, r, q, p)⋅C(p, i)⋅C(r, _d_1)))

In [15]:
df |> fc |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_d_3, _d_2) -> 
        (∑((_d, i, p, q), (4.0⋅C(q, i)'⋅J(_d, _d_3, q, p)⋅_k(_d, _d_2)⋅C(p, i)))+∑((_d, _d_1, q, r), (4.0⋅C(q, _d_2)'⋅J(_d, r, q, _d_3)⋅_k(_d, _d_1)⋅C(r, _d_1)))+∑((_d, _d_1, p, r), (4.0⋅J(_d, r, _d_3, p)⋅_k(_d, _d_1)⋅C(p, _d_2)⋅C(r, _d_1))))

## Foster Boys

In [16]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::Z) -> sum((p, q), R(p, q) * C(i, p)' * C(i, q))
        sum(i, sum((p, q), R2(p, q) * C(i, p)' * C(i, q)) - r(i)' * r(i))
    end); f

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                ∑((p, q), (C(i, p)'⋅R(p, q)⋅C(i, q)))
            ∑((i), ((-1.0⋅r(i)'⋅r(i))+∑((p, q), (C(i, p)'⋅R2(p, q)⋅C(i, q)))))
        end)

In [17]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(R, R2) -> 
    (C) -> 
        (_d, _d_1) -> 
            (∑((p), (2.0⋅R2(_d_1, p)⋅C(_d, p)))+∑((p, _p, q), (-4.0⋅C(_d, _p)'⋅R(_d_1, p)⋅R(_p, q)⋅C(_d, q)⋅C(_d, p))))

## Localized Wannier Functions

In [18]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (Z, Z, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (Z, Z, BZ) -> C
    end
end;

In [19]:
f = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::Z, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end)

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))
            ∑((n, b), (ρ(n, b)'⋅ρ(n, b)))
        end)

In [20]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = vdiff(eval_all(f); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            ∑((b, p, k, _p, q), (4.0⋅U(_p, _d_1, (b+k))'⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k)))

In [48]:
f, _ = @pct begin
    x = 1
    y = (m::R) -> m - 2
    (j::I) ->  x+ y(j)
end; f

let 
    x = 1
    y = (m) -> 
        (m+-2.0)
    (j) -> 
        (y(j)+x)
end

In [26]:
f, _ = @pct begin
    y = (m::R) -> m - 2
    (j::I ∈ (1, 2)) -> y(j)
end; f

let 
    y = (m) -> 
        (m+-2.0)
    (j ∈ (1, 2)) -> 
        y(j)
end